In [374]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#models
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
#metrics
import sklearn.metrics as metrics

#hiperparameters
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline  # Para construir el Pipeline
from sklearn.compose import ColumnTransformer  # Para aplicar transformaciones a diferentes columnas
from sklearn.preprocessing import FunctionTransformer 
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder

from sklearn.impute import SimpleImputer

pd.set_option('display.max_columns', 150) 
pd.set_option('display.max_rows', 300)

In [375]:
df_subvencions = pd.read_csv('./data/Subvencions_atorgades_AjuntBCN_i_Instituts.csv')

In [376]:
df_subvencions.head(1)

,Entitat_Municipal,Organ_Gestor,Beneficiari,NIF_CIF,Tipologia_De_Subvencio,Codi_De_Subvencio,Objecte,Data_Convocatoria,Data_Atorgament,Import_Sollicitat,Import_Total_Projecte,Import_Atorgat_Inicial,Import_Reintegrat_Total
0,Ajuntament de Barcelona,"Gerència Drets de Ciutadania, Participació i T...",ASSOCIACIO UCRAINESA DJERELO,G65874968,CONVOCATÒRIA GENERAL DE SUBVENCIONS 2019.DRETS...,19S00465,V Ucrania Fest,24.01.2019,16.09.2019,5000.0,14900.0,4500.0,0.0


In [377]:
df_subvencions.drop(['Objecte','Import_Reintegrat_Total','Beneficiari','NIF_CIF','Tipologia_De_Subvencio','Codi_De_Subvencio'], axis=1, inplace=True)

In [378]:
df_subvencions.head(1)

,Entitat_Municipal,Organ_Gestor,Data_Convocatoria,Data_Atorgament,Import_Sollicitat,Import_Total_Projecte,Import_Atorgat_Inicial
0,Ajuntament de Barcelona,"Gerència Drets de Ciutadania, Participació i T...",24.01.2019,16.09.2019,5000.0,14900.0,4500.0


In [379]:
# Convierto fecha a tipo date time
df_subvencions['Data_Convocatoria'] = pd.to_datetime(df_subvencions['Data_Convocatoria'], format='%d.%m.%Y', errors='coerce')
df_subvencions['Data_Atorgament'] = pd.to_datetime(df_subvencions['Data_Atorgament'], format='%d.%m.%Y', errors='coerce')

In [380]:
df_subvencions['diferencia_dias'] = df_subvencions['Data_Atorgament']-df_subvencions['Data_Convocatoria']
df_subvencions['diferencia_dias'] = df_subvencions['diferencia_dias'].dt.days

In [381]:
df_subvencions = df_subvencions[df_subvencions['diferencia_dias']>0]

In [382]:
df_subvencions['Organ_Gestor'].nunique()

33

In [383]:
df_subvencions['Organ_Gestor'] = df_subvencions['Organ_Gestor'].replace('gerencia municipal', 'gerència municipal')
df_subvencions['Organ_Gestor'] = df_subvencions['Organ_Gestor'].replace("gerència d'àrea d'agenda 2030, transició digital,", "gerència d'àrea d'agenda 2030, transició digital i esports")  

In [384]:
df_subvencions.drop(['Data_Convocatoria','Data_Atorgament'], axis=1, inplace=True)

In [385]:
df_subvencions.describe()

,Import_Sollicitat,Import_Total_Projecte,Import_Atorgat_Inicial,diferencia_dias
count,3.355900e+04,3.355900e+04,3.355900e+04,33559.000000
mean,8.063863e+03,2.446405e+04,5.466970e+03,227.064036
std,2.080629e+04,1.528183e+05,1.714821e+04,55.809468
min,0.000000e+00,0.000000e+00,2.500000e-01,2.000000
25%,1.000000e+03,2.000000e+03,6.000000e+02,182.000000
50%,3.000000e+03,6.500000e+03,1.800000e+03,231.000000
75%,7.488000e+03,1.850000e+04,4.000000e+03,260.000000
max,1.631954e+06,1.239800e+07,1.631954e+06,758.000000


In [386]:
df_subvencions = df_subvencions[df_subvencions['Import_Sollicitat']<=100000]
df_subvencions.shape

(33299, 6)

In [387]:
df_subvencions.sample(1)

,Entitat_Municipal,Organ_Gestor,Import_Sollicitat,Import_Total_Projecte,Import_Atorgat_Inicial,diferencia_dias
9174,Ajuntament de Barcelona,Gerència de Presidència i Economia,0.0,78.5,300.0,260.0


In [388]:
columnas_objeto = list(df_subvencions.describe(include=object)) 
columnas_numericas = list(df_subvencions.describe()) 

In [389]:
df_subvencions # dataset original

def clean_string(columna):
    """elimina espacios al final y al comienzo y convierte en minuscula"""
    return columna.apply(lambda x: x.str.strip().str.lower())

def fix_null(dataset):
    dataset['Organ_Gestor'] = dataset['Organ_Gestor'].fillna(dataset['Entitat_Municipal'])
    return dataset

In [390]:
columnas_numericas

['Import_Sollicitat',
 'Import_Total_Projecte',
 'Import_Atorgat_Inicial',
 'diferencia_dias']

In [401]:
preprocessor = ColumnTransformer(
    transformers=[
        ("imputar_nulos_constante_na", SimpleImputer(strategy="constant", fill_value="NA"), ['Organ_Gestor']),
        ('objeto', FunctionTransformer(clean_string), ['Entitat_Municipal','Organ_Gestor'])
    ]
)

In [402]:
onhot_scaler = ColumnTransformer(
    transformers = [
        ('ohe', OneHotEncoder(sparse_output=False), columnas_objeto),
        ('minmax_scale', MinMaxScaler(), ['Import_Sollicitat','Import_Atorgat_Inicial','diferencia_dias']),
    ]
)

In [403]:
pipeline = Pipeline(steps = [
    ("preprocesing", preprocessor),
    ("transform", onhot_scaler),
    ("modelo", DecisionTreeClassifier(random_state=12))
])

In [404]:
X = df_subvencions.drop('Import_Total_Projecte', axis=1)  # Características
y = df_subvencions['Import_Total_Projecte']  # Variable objetivo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
# Entreno el modelo con mi pipeline
pipeline.fit(X_train, y_train)